В этом задании вам предлагается реализовать такую демонстрацию для вашего алгоритма анализа тональности отзывов на фильмы, используя flask.
из прошлымх заданий (week 2) были определены модели и параметры. использую их + добавлю лематизацию

In [1]:
from nltk.corpus import movie_reviews, stopwords
from nltk.stem import WordNetLemmatizer
from nltk import word_tokenize
import string
from string import punctuation
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.linear_model import LogisticRegression,LogisticRegressionCV
from sklearn.model_selection import cross_val_score,StratifiedKFold
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split
import warnings
import pickle
warnings.filterwarnings('ignore')


In [2]:
negids = movie_reviews.fileids('neg')
posids = movie_reviews.fileids('pos')

In [3]:
negfeats = [movie_reviews.words(fileids=[f]) for f in negids]

In [4]:
posfeats = [movie_reviews.words(fileids=[f]) for f in posids]

In [5]:
texts = negfeats + posfeats
labels = [0] * len(negfeats) + [1] * len(posfeats)

In [6]:
len(posfeats)/len(texts)*100

50.0

приведем каждый текст в строку

In [7]:
new_texts = [" ".join(texts[i]) for i in range(len(texts))]

In [8]:
stop = stopwords.words('english')
lemmatizer = WordNetLemmatizer()


лематизируем

In [9]:
def delete_punctuation(line):
    translation_table = str.maketrans('', '', string.punctuation)
    return line.translate(translation_table).lower()

In [10]:
def format_str(line):
    word_list = word_tokenize(line)
    words = [lemmatizer.lemmatize(word) for word in word_list]
    stroka = delete_punctuation(' '.join(words))
    words = [word.strip() for word in stroka.split()]
    return ' '.join(words)

In [11]:
for i, line in enumerate(new_texts):
    new_texts[i] = format_str(line)

выделим холд аут выборку

In [12]:
X_train, X_test, y_train, y_test = train_test_split(new_texts,labels, test_size=0.1, random_state=123)

In [13]:
sum(y_test)

94

обучим векторайзер


In [14]:
vectorizer = CountVectorizer(ngram_range=(1,2), stop_words = stop)

In [15]:
vectorizer_for_cross = vectorizer.fit(X_train)

In [16]:
vector = vectorizer.fit_transform(X_train)

In [17]:
cross_val_score(LogisticRegression(random_state=122), vector, y_train,scoring='f1', cv=5).mean()

0.8404769363747933

порешаем вопросики с переобучением

In [18]:
#skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=122)

#c_values = np.logspace(-2, 3, 100)

#logit_searcher = LogisticRegressionCV(Cs=c_values, cv=skf, verbose=1 )
#logit_searcher.fit(vector.transform(X_train), y_train)
# logit_searcher.C_ - array([0.0225702])

In [19]:
C = 0.0225702

In [20]:
clf = LogisticRegression(C = C, random_state=122)

In [21]:
clf.fit(vector, y_train)

LogisticRegression(C=0.0225702, class_weight=None, dual=False,
                   fit_intercept=True, intercept_scaling=1, l1_ratio=None,
                   max_iter=100, multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=122, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)

In [22]:
sum(clf.predict(vectorizer_for_cross.transform(X_test)) == y_test) / len (y_test)

0.835

Обучиним итоговый векторайзер

In [23]:
vectorizer = CountVectorizer(ngram_range=(1,2), stop_words = stop)

In [24]:
vector_itog = vectorizer.fit(new_texts)

In [25]:
with open("./BigramUnprocessedVectorizer.pkl","wb") as f:
    pickle.dump(vector_itog, f)

обучим классификатор

In [26]:
clf_itog = LogisticRegression(C=C,random_state=122)

In [27]:
clf_itog.fit(vector_itog.transform(new_texts), labels)

LogisticRegression(C=0.0225702, class_weight=None, dual=False,
                   fit_intercept=True, intercept_scaling=1, l1_ratio=None,
                   max_iter=100, multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=122, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)

In [28]:
with open("./DefaultLogisticBigramUnprocessedTextSentiment.pkl","wb") as f:
    pickle.dump(clf_itog, f)

In [297]:
def delete_punctuation(line):
    translation_table = str.maketrans('', '', string.punctuation)
    return line.translate(translation_table).lower()


def format_str(line):
    word_list = word_tokenize(line)
    words = [lemmatizer.lemmatize(word) for word in word_list]
    stroka = delete_punctuation(' '.join(words))
    words = [word.strip() for word in stroka.split()]
    return [' '.join(words)]

In [310]:
vector_itog.transform(format_str('gkgk fk fk'))

<1x544987 sparse matrix of type '<class 'numpy.int64'>'
	with 0 stored elements in Compressed Sparse Row format>

In [32]:
clf_itog.predict_proba(vector_itog.transform([format_str(a)]))

array([[0.70057863, 0.29942137]])

In [29]:
a = 'clf_itog.predict(vector_itog.transform(format_str(''Everyone plays their part pretty well in this "little nice movie". Belushi gets the chance to live part of his life differently, but ends up realizing that what he had was going to be just as good or maybe even better. The movie shows us that we ought to take advantage of the opportunities we have, not the ones we do not or cannot have. If U can get this movie on video for around $10, it\xc2\xb4d be an investment!'')))[0]'